In [176]:
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

In [177]:
import requests
from bs4 import BeautifulSoup


class BaseLoader:
    def __init__(self, urls):
        self.urls = urls

    def load(self):
        references = []
        for url in self.urls:
            response = requests.get(url, verify=False)
            soup = BeautifulSoup(response.text, 'html.parser')
            references.append(soup.get_text())
        
        return references

In [178]:
with open("hypos.txt") as f:
    hypos = [line.strip() for line in f]

with open("refs.txt") as f:
    refs = [line.strip() for line in f]

In [179]:
refs

['https://ts.hust.edu.vn/training-cate/nganh-dao-tao-dai-hoc/dien-tu-va-vien-thong',
 'https://huongnghiep.hocmai.vn/chuyen-nganh/ky-thuat-dien-tu-vien-thong/dai-hoc-bach-khoa-ha-noi/']

In [180]:
loader = BaseLoader(refs)
refs= loader.load()

In [181]:
# Using textwrap to wrap the text
import textwrap

for ref in refs:
    print("\n".join(textwrap.wrap(ref, width=120)))
    print("\n")

   Điện tử và Viễn thông                                      Thông tin tuyển sinh Đại học Bách khoa Hà Nội HUST
ADMISSIONS INFORMATION         Đăng nhập  Đơn vị liên quan           Home     Hướng nghiệp     Tuyển sinh Đại học
Thông tin chung     Chương trình đào tạo đại học     Xét tuyển tài năng     Kỳ thi đánh giá tư duy     Xác thực chứng
chỉ Ngoại ngữ     Điểm chuẩn các năm       Tuyển sinh Sau đại học     Tuyển sinh cao học     Đăng ký thạc sĩ online
Ngành đào tạo thạc sĩ       Tuyển sinh nghiên cứu sinh     Mẫu hồ sơ Nghiên cứu sinh         International Admissions
Học phí - Học bổng     Học phí     Học bổng       Hỏi đáp     Liên hệ     Ban TS-HN     Đơn vị đào tạo
Tổng quan Chương trình đào tạo Học bổng Cơ hội việc làm Đơn vị quản lý       Trang chủ  Ngành đào tạo ĐẠI HỌC  Điện tử
và Viễn thông     Tổng quan    Ngôn ngữ đào tạo: Tiếng Việt Mã xét tuyển: ET1   Xét tuyển tài năng:  Xét tuyển bằng Giải
thưởng HSG QG-QT/Chứng chỉ Quốc tế/HSNL   Xét tuyển theo KQ Kỳ thi ĐGTD:
Tổ hợ

In [182]:
# Normalizing Vietnamese text
import unicodedata
import re

def normalize_text(text):
    text = unicodedata.normalize("NFC", text)
    text = re.sub(r"\s+", " ", text)
    return text

In [183]:
refs = [' '.join(normalize_text(ref) for ref in refs)]
hypos = [' '.join(normalize_text(hypo) for hypo in hypos)]

### Using BERTScore for Evaluation

In [184]:
from bert_score import score

In [185]:
# Hide the loading messages
import logging
import transformers
transformers.tokenization_utils.logger.setLevel(logging.ERROR)
transformers.configuration_utils.logger.setLevel(logging.ERROR)
transformers.modeling_utils.logger.setLevel(logging.ERROR)

In [186]:
P, R, F1 = score(hypos, refs, lang="vi", verbose=False)

In [187]:
print(f"BERT score: Precision={P.mean():.2f}, Recall={R.mean():.2f}, F1={F1.mean():.2f}")

BERT score: Precision=0.73, Recall=0.69, F1=0.71


#### F1 score is 0.71 that is quite good for my project!!!

### Using ROUGE score for Evaluation

In [188]:
from rouge_score import rouge_scorer

def rouge_score(hypos, refs):
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    scores = [scorer.score(ref, hypo) for ref, hypo in zip(refs, hypos)]
    return scores

rouge = rouge_score(hypos, refs)

In [190]:
print("ROUGE scores:")
for i, score in enumerate(rouge):
    print(f"ROUGE1: Precision={score['rouge1'].precision:.2f}, Recall={score['rouge1'].recall:.2f}, F1={score['rouge1'].fmeasure:.2f}")
    print(f"ROUGE2: Precision={score['rouge2'].precision:.2f}, Recall={score['rouge2'].recall:.2f}, F1={score['rouge2'].fmeasure:.2f}")
    print(f"ROUGEL: Precision={score['rougeL'].precision:.2f}, Recall={score['rougeL'].recall:.2f}, F1={score['rougeL'].fmeasure:.2f}")

ROUGE scores:
ROUGE1: Precision=0.94, Recall=0.18, F1=0.31
ROUGE2: Precision=0.83, Recall=0.16, F1=0.27
ROUGEL: Precision=0.67, Recall=0.13, F1=0.22
